In [6]:
# pip install yfinance pandas

import yfinance as yf
import pandas as pd

FX_TICKERS = {
    "USDINR": "INR=X",
    "EURINR": "EURINR=X",
    "GBPINR": "GBPINR=X",
    "JPYINR": "JPYINR=X",
    "EURUSD": "EURUSD=X",
    # add more here...
}

In [7]:
def fetch_fx_timeseries(
    pairs,
    start="2020-01-01",
    end=None,
    interval="1d",
    auto_adjust=False,
):
    """
    Fetch FX OHLCV time series from Yahoo Finance via yfinance.

    pairs: str (e.g., "USDINR") or list/tuple of str (e.g., ["USDINR","EURINR"])
    Returns a tidy DataFrame with columns:
      date, pair, open, high, low, close, adj_close, volume
    """
    # ---- Normalize pairs input (single -> list) ----
    if isinstance(pairs, str):
        pairs = [pairs]
    elif isinstance(pairs, (list, tuple, set)):
        pairs = list(pairs)
    else:
        raise TypeError("pairs must be a string (single pair) or a list/tuple/set of pairs.")

    if not pairs:
        raise ValueError("pairs cannot be empty.")

    # ---- Map to Yahoo tickers ----
    tickers = []
    ticker_to_pair = {}

    for p in pairs:
        p = p.strip().upper()
        if p not in FX_TICKERS:
            raise ValueError(f"Unknown pair '{p}'. Available: {sorted(FX_TICKERS.keys())}")
        t = FX_TICKERS[p]
        tickers.append(t)
        ticker_to_pair[t] = p

    # ---- Download (one call handles both single & multi) ----
    raw = yf.download(
        tickers=" ".join(tickers),
        start=start,
        end=end,
        interval=interval,
        group_by="ticker",
        auto_adjust=auto_adjust,
        threads=True,
        progress=False,
    )

    if raw is None or raw.empty:
        raise RuntimeError("No data returned. Check tickers/dates/interval.")

    # ---- Convert to tidy format ----
    frames = []

    if isinstance(raw.columns, pd.MultiIndex):
        # Multiple tickers: columns are (TICKER, Field)
        for t in tickers:
            if t not in raw.columns.get_level_values(0):
                continue
            df = raw[t].copy()
            df.columns = [c.lower().replace(" ", "_") for c in df.columns]
            df = df.reset_index()
            df.insert(1, "pair", ticker_to_pair[t])
            frames.append(df)
    else:
        # Single ticker: columns are Field only
        df = raw.copy()
        df.columns = [c.lower().replace(" ", "_") for c in df.columns]
        df = df.reset_index()
        df.insert(1, "pair", ticker_to_pair[tickers[0]])
        frames.append(df)

    out = pd.concat(frames, ignore_index=True)

    # Standardize column name for date index (can be Date or Datetime depending on interval)
    if "date" not in out.columns:
        if "Date" in out.columns:
            out = out.rename(columns={"Date": "date"})
        elif "Datetime" in out.columns:
            out = out.rename(columns={"Datetime": "date"})
        else:
            # if reset_index produced something else
            out = out.rename(columns={out.columns[0]: "date"})

    expected = ["date", "pair", "open", "high", "low", "close", "adj_close", "volume"]
    for c in expected:
        if c not in out.columns:
            out[c] = pd.NA

    out = out[expected].sort_values(["pair", "date"]).reset_index(drop=True)
    return out

In [8]:
if __name__ == "__main__":
    # ✅ Single pair
    pair = "EURUSD"
    single_df = fetch_fx_timeseries(pair, start="2020-01-01")
    print("Single pair rows:", len(single_df))
    single_df.to_csv("{}_from_2020.csv".format(pair), index=False)

    # ✅ Multiple pairs
    #multi_df = fetch_fx_timeseries(["USDINR", "EURINR", "GBPINR"], start="2020-01-01")
    #print("Multiple pairs rows:", len(multi_df))
    #multi_df.to_csv("fx_multi_from_2020.csv", index=False)


Single pair rows: 1582
